In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[4]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

Loading spark-stubs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/03/27 13:52:14 INFO SparkContext: Running Spark version 2.4.5
23/03/27 13:52:17 INFO SparkContext: Submitted application: 9ba1dc6c-3ec3-43eb-9298-850adcd77587
23/03/27 13:52:17 INFO SecurityManager: Changing view acls to: Alvaro
23/03/27 13:52:17 INFO SecurityManager: Changing modify acls to: Alvaro
23/03/27 13:52:17 INFO SecurityManager: Changing view acls groups to: 
23/03/27 13:52:17 INFO SecurityManager: Changing modify acls groups to: 
23/03/27 13:52:17 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(Alvaro); groups with view permissions: Set(); users  with modify permissions: Set(Alvaro); groups with modify permissions: Set()
23/03/27 13:52:20 INFO Utils: Successfully started service 'sparkDriver' on port 56381.
23/03/27 13:52:21 INFO SparkEnv: Registering MapOutputTracker
23/03/27 13:52:21 INFO SparkEnv: Registering BlockManagerMa

23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-cache_2.12/1.1.0-M13-1/coursier-cache_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:56381/jars/coursier-cache_2.12-1.1.0-M13-1.jar with timestamp 1679917942109
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-core_2.12/1.1.0-M13-1/coursier-core_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:56381/jars/coursier-core_2.12-1.1.0-M13-1.jar with timestamp 1679917942148
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier_2.12/1.1.0-M13-1/coursier_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:56381/jars/coursier_2.12-1.1.0-M13-1.jar with timestamp 1679917942149
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppDa

...

23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/javax/xml/stream/stax-api/1.0-2/stax-api-1.0-2.jar at spark://LAPTOP-30N4Q54J:56381/jars/stax-api-1.0-2.jar with timestamp 1679917942520
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.7.45/RoaringBitmap-0.7.45.jar at spark://LAPTOP-30N4Q54J:56381/jars/RoaringBitmap-0.7.45.jar with timestamp 1679917942521
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/codehaus/jackson/jackson-jaxrs/1.9.13/jackson-jaxrs-1.9.13.jar at spark://LAPTOP-30N4Q54J:56381/jars/jackson-jaxrs-1.9.13.jar with timestamp 1679917942522
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/dropwizard/metrics/metrics-js

23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/arrow/arrow-memory/0.10.0/arrow-memory-0.10.0.jar at spark://LAPTOP-30N4Q54J:56381/jars/arrow-memory-0.10.0.jar with timestamp 1679917942618
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-shuffle/2.6.5/hadoop-mapreduce-client-shuffle-2.6.5.jar at spark://LAPTOP-30N4Q54J:56381/jars/hadoop-mapreduce-client-shuffle-2.6.5.jar with timestamp 1679917942618
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.7.9/jackson-module-paranamer-2.7.9.jar at spark://LAPTOP-30N4Q54J:56381/jars/jackson-module-paranamer-2.7.9.jar with timestamp 1679917942619
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/U

23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-util/9.4.15.v20190215/jetty-util-9.4.15.v20190215.jar at spark://LAPTOP-30N4Q54J:56381/jars/jetty-util-9.4.15.v20190215.jar with timestamp 1679917942724
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.12/2.3.3/shapeless_2.12-2.3.3.jar at spark://LAPTOP-30N4Q54J:56381/jars/shapeless_2.12-2.3.3.jar with timestamp 1679917942724
23/03/27 13:52:22 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.4.0/spark-stubs_24_2.12-0.4.0.jar at spark://LAPTOP-30N4Q54J:56381/jars/spark-stubs_24_2.12-0.4.0.jar with timestamp 1679917942726
23/03/27 13:52:23 INFO Executor: Starting executor ID driver on host localhost
23/03/27 13:52:23 INFO Executor: Using REPL 

Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@6adb4bdc
import spark.implicits._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}



In [2]:
import org.apache.spark.sql.expressions.Window

val data = spark.read.parquet("D:/TFGData/dayParquet/")
val stations = spark.read.option("delimiter", ";").csv("D:/TFGData/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

val window = Window.partitionBy($"indicativo", $"año").orderBy($"fecha")

val results = data
    .filter(!func.isnull($"tmax") && $"tmax" >= 40)
    .withColumn("año", func.year($"fecha"))
    .withColumn("n_fila", func.row_number().over(window))
    .withColumn("id", func.expr("date_sub(fecha, n_fila)"))
    .groupBy($"indicativo", $"año", $"id")
    .agg(func.count($"id").alias("dias"), func.avg($"tmax"), func.max($"tmax"), func.min($"tmax"))
    .filter($"dias" > 3)
    .join(stations, "indicativo")
    .select($"ubicacion", $"provincia", $"año", $"dias", func.round($"avg(tmax)", 2).alias("avg(tmax)"), 
            $"max(tmax)", $"min(tmax)")

parquet at cmd1.sc:3

291 / 291

parquet at cmd1.sc:3

1 / 1

csv at cmd1.sc:4

1 / 1

import org.apache.spark.sql.expressions.Window


data: DataFrame = [fecha: date, nombre: string ... 18 more fields]
stations: DataFrame = [provincia: string, indicativo: string ... 1 more field]
window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@79d5df6d
results: DataFrame = [ubicacion: string, provincia: string ... 5 more fields]

In [3]:
val resultsSave = results
    .groupBy($"provincia", $"año")
    .agg(func.count($"provincia"), func.avg($"dias"), func.avg($"avg(tmax)"), func.max($"max(tmax)"), func.min($"min(tmax)"))
    .select($"provincia", $"año", $"count(provincia)".alias("nº de olas de calor"), 
            $"avg(dias)".alias("duracion media"), $"avg(avg(tmax))".alias("avg(tmax)"), 
            $"max(max(tmax))".alias("max(tmax)"), $"min(min(tmax))".alias("min(tmax)"))
    
resultsSave
    .withColumnRenamed("nº de olas de calor", "nOlasCalor")
    .withColumnRenamed("duracion media", "duracionMedia")
    .withColumnRenamed("avg(tmax)", "avgTmax")
    .withColumnRenamed("max(tmax)", "maxTmax")
    .withColumnRenamed("min(tmax)", "minTmax")
    .write.format("parquet").partitionBy("año").mode("overwrite").save("D:/borrar/")

run at ThreadPoolExecutor.java:1128

1 / 1

save at cmd2.sc:14

193 / 193

save at cmd2.sc:14

64 / 64

save at cmd2.sc:14

64 / 64

resultsSave: DataFrame = [provincia: string, año: int ... 5 more fields]

In [ ]:
results
    .withColumnRenamed("dias", "duracion (dias)")
    .withColumnRenamed("avg(tmax)", "temperatura media")
    .withColumnRenamed("max(tmax)", "temperatura maxima")
    .withColumnRenamed("min(tmax)", "temperatura minima")
    .orderBy($"ubicacion", $"año")
    .show()

In [ ]:
results
    .groupBy($"año")
    .agg(func.count($"año").alias("nº olas de calor"))
    .orderBy($"año")
    .show()

Para observar los resultados de forma gráfica en el mapa acceder al notebook `Olas de calor - Representación en mapa`